# 식품의약품안전처_의료기기 동등제품정보 (식의약헬스종합정보)
---
- 작성자 : 안지은
- 실행일 : 2023-01-10

In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warningsㄴ
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'

In [2]:
# 총 데이터 수 구하는 함수
def get_total_cnt(key):
    url = 'https://apis.data.go.kr/1471000/MdeqEqvlPrdtInfoService02/getMdeqEqvlPrdtInfoInq02?serviceKey=%s&type=json'%(key)
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    cnt = int(json_ob['body']['totalCount'])
    return cnt

In [3]:
# api 컨텐츠(json - data) 받아오는 함수
def get_api_conts(key,page):
    
    url = 'https://apis.data.go.kr/1471000/MdeqEqvlPrdtInfoService02/getMdeqEqvlPrdtInfoInq02?serviceKey=%s&pageNo=%s&numOfRows=100&type=json'%(key,str(page))
    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['body']['items']
    
    return data

### API 가져오기
---
- 한 페이지 당  100개 씩 제한
- 총 13391개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [4]:
equi_device = []
cnt = get_total_cnt(key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts(key,i)
    for item in js:
        equi_device.append(item['item'])


100%|██████████| 16/16 [00:13<00:00,  1.17it/s]


### 데이터 프레임 변환
---
- 컬럼명 : API 목록

In [5]:
equi_device_df = pd.json_normalize(equi_device)
equi_device_df.columns = ['동등제품제조수입구분','동등제품허가년','동등제품허가번호',
'동등제품허가일련번호','동등공고번호','동등공고품목명','동등공공유형번호','동등공고날짜',
'동일성제품여부','동일성제품허가번호','동등공고제품','의료기기품목번호','기관지역구분','업종',
'허가신고구분','의료기기품목허가번호','품목명','허가일자','취소취하구분','취소취하일자','상품명']
equi_device_df

,동등제품제조수입구분,동등제품허가년,동등제품허가번호,동등제품허가일련번호,동등공고번호,동등공고품목명,동등공공유형번호,동등공고날짜,동일성제품여부,동일성제품허가번호,...,의료기기품목번호,기관지역구분,업종,허가신고구분,의료기기품목허가번호,품목명,허가일자,취소취하구분,취소취하일자,상품명
0,제조업,1988,제인 88-19 호,198800121,None,None,None,None,N,None,...,198800121,의료기기정보기술지원센터,제조업,인증,제인 88-19 호,범용주입-배액용튜브카테터,1988-10-24,None,None,Penrose
1,제조업,1994,제인 94-25 호,199400294,None,None,None,None,N,None,...,199400294,의료기기정보기술지원센터,제조업,인증,제인 94-25 호,일회용기관절개튜브,1994-05-31,None,None,"SEWOON Teflon Tracheostomy Double Tube, SEWOON..."
2,제조업,1995,제허 95-13 호,199500073,None,None,None,None,N,None,...,199500073,식약처,제조업,허가,제허 95-13 호,매일착용소프트콘택트렌즈,1995-12-01,None,None,eyelike TORI-CON 외 1건
3,제조업,1997,제허 97-83 호,199700280,None,None,None,None,N,None,...,199700280,식약처,제조업,허가,제허 97-83 호,매일착용소프트콘택트렌즈,1997-10-18,None,None,"글램마돈나, 네오팝, 릴리브 그레이 (LYLIBE GRAY), 릴리브 브라운 (LY..."
4,제조업,1997,제허 97-86 호,199700283,None,None,None,None,N,None,...,199700283,식약처,제조업,허가,제허 97-86 호,매일착용소프트콘택트렌즈,1997-10-18,None,None,"달리, 달리Ⅱ, 달리엑스트라, 달리엑스트라Ⅱ, 모네, 루비퀸, 샤갈, 아이팝, 스모..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,제조업,2013,제인 13-2260 호,2013010815,None,None,None,None,N,None,...,2022006486,의료기기정보기술지원센터,제조업,인증,제인 22-4992 호,주사기,2022-12-14,None,None,"PRIME SYRINGE, MIRA LINE, ANGEL SYRINGE"
1502,제조업,2013,제인 13-2260 호,2013010815,None,None,None,None,N,None,...,2022006487,의료기기정보기술지원센터,제조업,인증,제인 22-4993 호,주사기,2022-12-14,None,None,None
1503,수입업,2021,수인 21-4389 호,2021004419,None,None,None,None,N,None,...,2022006638,의료기기정보기술지원센터,수입업,인증,수인 22-4600 호,치과용임플란트상부구조물,2022-12-23,None,None,NUVO Cover Screw & Healing Abutment
1504,수입업,2022,수허 22-149 호,2022003563,None,None,None,None,N,None,...,2022006762,식약처,수입업,허가,수허 22-300 호,다초점인공수정체,2022-12-30,None,None,None
